### Orion Peeters

#### 208409565

In [15]:
from google.colab import drive
import sys

drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/course 22969')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import torch
import tiktoken
import textwrap
import os
import random

# I importing the architecture and tools from the original model.
# The .py file is in the same folder in the repo, in order for the import to work please keep it that way
from gpt_core import (
    GPTModel,
    BASE_CONFIG,
    generate,
    text_to_token_ids,
    token_ids_to_text,
    format_input
)

#### **Important Prerequisite:**

The SFT model weights used in this part are not included in the repository
because they exceed GitHubs file size limits.

To run this code successfully, the user must first execute the code from Part 2
(available in this repository). Running Part 2 will train the model and save
the required weight file locally.

After the weights are generated and saved, you will be able to load them here.

In [17]:
# Setup Hardware
def load_finetuned_model():
    """
    Initializes the system and loads the specific SFT weights.
    """
    print("\n")
    print("INITIALIZING DEBATE SYSTEM")

    # Auto-detecting device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Hardware Device: {device}")

    # Load Tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Load Model Architecture
    print("Building GPT-2 Medium Architecture...")
    model = GPTModel(BASE_CONFIG)

    # Load Fine-Tuned Weights
    # The user will need to change the path here to his one
    model_path = "/content/drive/MyDrive/course 22969/gpt2/gpt2-medium355M-sft.pth"

    if os.path.exists(model_path):
        print(f"Loading weights from: {model_path}")
        try:
            state_diction = torch.load(model_path, map_location=device)

            # Removing '_orig_mod.' prefix added by torch.compile in part 2 code I added
            new_state_diction = {}
            for k, v in state_diction.items():
                if k.startswith("_orig_mod."):
                    new_key = k.replace("_orig_mod.", "")
                    new_state_diction[new_key] = v
                else:
                    new_state_diction[k] = v

            # Loading the cleaned dictionary
            model.load_state_dict(new_state_diction)
            print("Success: Model loaded.")

        except Exception as e:
            print(f"Error loading weights: {e}")
            return None, None, None
    else:
        print(f"Error: File not found at {model_path}")
        return None, None, None

    model.to(device)
    model.eval() # Setting to evaluation mode

    return model, tokenizer, device

In [18]:
class DebateAgent:
    """
    Represents a single debater in the arena.
    This class wraps the GPT model with a specific 'persona' and manages
    the generation process.
    """
    def __init__(self, name, persona, start_phrases, model, tokenizer, device):

        self.name = name
        self.persona = persona
        self.start_phrases = start_phrases
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.last_reply = "" # Memory buffer to check for repeated outputs

    def reply(self, history, round_num):
        """
        Generates a response based on the conversation history.
        This method implements the next mechanism:
        1. It attempts to generate a response.
        2. If the response is identical to the previous turn or too short, it retries.
        3. On retries, it increases the 'temperature' (randomness) to force a new path.
        """
        max_retries = 3

        for attempt in range(max_retries):
            # Picking a Random Opener
            # I strictly force the model to start with a specific phrase.
            # This prevents "Agreement bias" where the model just says "I agree."
            current_opener = random.choice(self.start_phrases)

            # Formating History
            recent = history[-2:]
            history_str = "\n".join([f"{msg['speaker']}: {msg['content']}" for msg in recent])

            # Building Prompt
            # preventing the model from getting stuck in a cache loop.
            entry = {
                "instruction": f"You are {self.name}. {self.persona} Do not repeat yourself. Be specific and detailed.",
                "input": f"ROUND: {round_num} (Attempt {attempt})\nCONTEXT:\n{history_str}"
            }

            full_prompt = format_input(entry) + current_opener

            # Generating
            current_temp = 0.9 + (attempt * 0.1)

            input_ids = text_to_token_ids(full_prompt, self.tokenizer).to(self.device)

            out_ids = generate(
                model=self.model,
                idx=input_ids,
                max_new_tokens=110,
                context_size=1024,
                temperature=current_temp,
                top_k=60,
                eos_id=50256
            )

            # Decoding and Cleaning
            full_text = token_ids_to_text(out_ids, self.tokenizer)

            # Slicing off the prompt to get just the new content
            raw_completion = full_text[len(full_prompt):].split("###")[0].strip()

            final_response = f"{current_opener} {raw_completion}"

            # If the response is exactly the same as the last round, or garbage (too short),
            # I skip 'return' and trigger the loop to try again with higher temperature.
            if final_response == self.last_reply or len(raw_completion) < 10:
                continue
            else:
                self.last_reply = final_response
                return final_response

        # Fallback - If we fail 3 times, return whatever we generated last to keep the code running.
        return final_response

In [29]:
class DebateArena:
    """
    Orchestrates the multi-agent debate simulation.

    This class acts as the 'System Controller'. It is responsible for:
    1. Managing the shared conversation history (Context Window).
    2. Alternating turns between the two DebateAgents.
    3. Displaying the real-time output in a readable format.
    """
    def __init__(self, topic, agent_a, agent_b):

        self.topic = topic
        self.agents = [agent_a, agent_b]

        # Shared Memory: This list grows as the debate progresses.
        # Both agents read from this list to understand the context.
        self.history = []
        self.round_num = 0

    def start_debate(self, rounds=3):
        """
        Runs the main execution loop of the debate.
        """
        print(f"\n TOPIC: '{self.topic}'")
        print("="*60)

        # Seed the Conversation
        # We need a "Moderator" entry to give the first agent something to reply to.
        seed = f"The topic is: {self.topic}. {self.agents[0].name}, go."
        self.history.append({"speaker": "Moderator", "content": seed})

        # Main Game Loop
        for r in range(rounds):
            self.round_num += 1
            print(f"--- ROUND {self.round_num} ---")

            # Iterate through agents
            for agent in self.agents:

                # Generate Reply
                response = agent.reply(self.history, self.round_num)

                # Update Shared Memory
                self.history.append({"speaker": agent.name, "content": response})

                # Print Output
                print(f"\n🗣️  {agent.name.upper()}:")
                print(textwrap.fill(response, width=70))

            print("")

        print("="*60)

#### The Test Case

I choose this specific topic as a test case. The user can change topic, persona and openers if he wants to try and genrate an argument regarding a different subject

In [46]:
if __name__ == "__main__":
    """
    This block sets up the simulation parameters and initiates the debate.
    It demonstrates the core innovation of the project: 'Biased Opener Injection'.

    By pre-defining the first few words of every sentence, I force the
    underlying language model (which inherently wants to agree/be polite)
    to argue.
    """

    # Loading the fine-tuned system
    model, tokenizer, device = load_finetuned_model()

    if model:
        # These are the "keys" to the debate.
        elon_openers = [
            "I fundamentally believe that pausing AI would be a disaster because",
            "The data clearly shows that AI is already saving lives by",
            "If we stop progress now, we will lose the chance to",
            "We cannot let fear stop us when the benefits are",
            "The reality is that AI regulation will only serve to"
        ]

        sam_openers = [
            "However, we must consider the catastrophic risk that",
            "But the danger is that unchecked systems will",
            "You are ignoring the critical fact that AI models are",
            "History warns us that powerful technology without safety leads to",
            "We simply cannot control a superintelligence that is"
        ]

        # I created two distinct agents using the same underlying GPT-2 model.
        # The 'persona' string sets the system instruction (Context).
        # The 'start_phrases' force the output alignment.

        optimist = DebateAgent(
            name="Elon",
            persona="You are a Techno-Optimist. You argue that AI is safe and essential.",
            start_phrases=elon_openers,
            model=model, tokenizer=tokenizer, device=device
        )

        skeptic = DebateAgent(
            name="Sam",
            persona="You are an AI Safety Researcher. You argue that AI is an existential threat.",
            start_phrases=sam_openers,
            model=model, tokenizer=tokenizer, device=device
        )

        # STARTING THE SIMULATION
        arena = DebateArena(
            topic="Should AI be paused?",
            agent_a=optimist,
            agent_b=skeptic
        )

        arena.start_debate(rounds=3)



INITIALIZING DEBATE SYSTEM
Hardware Device: cuda
Building GPT-2 Medium Architecture...
Loading weights from: /content/drive/MyDrive/course 22969/gpt2/gpt2-medium355M-sft.pth
Success: Model loaded.

 TOPIC: 'Should AI be paused?'
--- ROUND 1 ---

🗣️  ELON:
We cannot let fear stop us when the benefits are so obvious.

🗣️  SAM:
You are ignoring the critical fact that AI models are in fact an
existential threat. Keep this in mind when trying to reach consensus.

--- ROUND 2 ---

🗣️  ELON:
I fundamentally believe that pausing AI would be a disaster because it
would remove an essential capability from the world.

🗣️  SAM:
We simply cannot control a superintelligence that is changing the
rules of the game.

--- ROUND 3 ---

🗣️  ELON:
We cannot let fear stop us when the benefits are so enormous!

🗣️  SAM:
But the danger is that unchecked systems will soon affect every aspect
of our lives, from our daily lives to our futures.

